# Script for accessing OpenAI API to generate Showerthoughts

In [72]:
import os
import openai
import json
import pandas as pd

In [7]:
openai.organization = os.getenv("OPENAI_ORG_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [14]:
openai.Model.list()

<OpenAIObject list at 0x11701ce30> JSON: {
  "object": "list",
  "data": [
    {
      "id": "whisper-1",
      "object": "model",
      "created": 1677532384,
      "owned_by": "openai-internal",
      "permission": [
        {
          "id": "modelperm-KlsZlfft3Gma8pI6A8rTnyjs",
          "object": "model_permission",
          "created": 1683912666,
          "allow_create_engine": false,
          "allow_sampling": true,
          "allow_logprobs": true,
          "allow_search_indices": false,
          "allow_view": true,
          "allow_fine_tuning": false,
          "organization": "*",
          "group": null,
          "is_blocking": false
        }
      ],
      "root": "whisper-1",
      "parent": null
    },
    {
      "id": "babbage",
      "object": "model",
      "created": 1649358449,
      "owned_by": "openai",
      "permission": [
        {
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "object": "model_permission",
          "created": 16690855

In [140]:
number_of_showerthoughts = 100
prompt = f"Please generate {number_of_showerthoughts} showerthoughts, which are inspired by the Reddit community r/showerthoughts. "+\
        "Do not repeat identical syntax between different sentences while only exchanging a single or few words. "+\
        "Be clever, creative, and (if suitable for the respective topic) funny. "+\
        "The showerthoughts should be relatable and connected to things that people might encounter during mundane tasks."


In [ ]:
# Our model: "gpt-3.5-turbo"
# This model has a context limit of 4k tokens. 
# Generating 100 Showerthoughts requires ~2-2.5k

results_list = []

while len(results_list)<20:
    results_list.append(openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
      ]
    ))

In [26]:
len(results_list)

5

In [66]:
results_list[19]['choices'][0]['message']['content']

'1. Are oranges called oranges because they\'re orange, or is the color orange called orange because of the fruit?\n\n2. If you never learned how to read, you wouldn\'t be able to understand anything I\'m saying, even if I spoke your language.\n\n3. Sleeping is so refreshing, even though it\'s just lying down and doing nothing for hours.\n\n4. Why do we call it "taking a dump" when we\'re actually leaving it behind?\n\n5. It\'s weird that we feel the need to apologize to inanimate objects when we accidentally hit them with our body parts.\n\n6. What if "yo mama" jokes were invented by moms to get their kids to roast them so they could seem cool?\n\n7. Is it just me, or does the sound of rain hitting the roof make everything feel more peaceful?\n\n8. If you think about it, we\'re just collections of atoms that have somehow gained consciousness.\n\n9. Ketchup is basically just wet tomato fruit salad.\n\n10. If a pizza has less toppings on one side, does that make it half a pizza or just 

# Convert to DataFrame

In [111]:
results_list_df = pd.DataFrame(results_list)

In [67]:
# Access the raw text from GPT Output and collect in a single list
# dict_keys(['id', 'object', 'created', 'model', 'choices', 'usage'])

raw_showerthoughts = []
counter = 0
for r in results_list:
    raw_showerthoughts.extend(counter, r['choices'][0]['message']['content'].replace("\n\n", "\n").split("\n"))

In [119]:
# Convert to DataFrame
showerthoughts_df = pd.DataFrame(raw_showerthoughts)

In [133]:
# Identify rows that are not showerthoughts (due to not starting with a number):
showerthoughts_df_clean = showerthoughts_df.drop(
    showerthoughts_df.loc[showerthoughts_df[0].apply(lambda x: x[0].isalpha() == True)].index)\
    .reset_index().drop("index", axis=1)


In [134]:
showerthoughts_df_clean

,0
0,"1. If we break down the word ""breakfast,"" tech..."
1,2. The first person to try seafood must have g...
2,3. We live in a world where we have pocket-siz...
3,4. The only part of your reflection that you c...
4,"5. If you think about it, a traffic jam is jus..."
...,...
1995,"96. Why do we always say ""I'm fine"" even when ..."
1996,97. It's funny how we can feel the sun on our ...
1997,98. If we could turn back time like in Cher's ...
1998,"99. Why do we always say ""the grass is always ..."


# Save outputs

In [78]:
rl_path = "results_list_df_2.pkl"
results_list_df.to_pickle(rl_path)

In [136]:
st_path = "showerthoughts_df_2.pkl"
showerthoughts_df_clean.to_pickle(st_path)

# Testing

In [15]:
# Our model: "gpt-3.5-turbo"
# This model has a context limit of 4k tokens. 
# Generating 100 Showerthoughts requires ~2-2.5k

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
  ]
)


In [16]:
print(completion)

{
  "id": "chatcmpl-7UZOM9G1I4n9p1As9zBlKiTIKIURZ",
  "object": "chat.completion",
  "created": 1687520242,
  "model": "gpt-3.5-turbo-0301",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "1. You always wear clothes, but you'll never know their true colors until you see them under different lighting conditions.\n2. Pretending to be asleep is the human version of \"I'm not touching you.\"\n3. Toilet paper is a perfect example of something that you never appreciate until you run out.\n4. Eating breakfast food for dinner is the ultimate act of rebellion against societal norms.\n5. The only place where the phrase \"time flies\" actually makes sense is inside an airplane.\n6. The biggest plot twist in life is realizing that the duck, duck, goose game has no real winner.\n7. You never truly appreciate your bed until you've spent the whole day standing up.\n8. The invention of the spoon was a game changer because it allowed people to 

# Prompt Graveyard

In [ ]:
prompt_v1 = "Please generate 100 showerthoughts, which are inspired by the Reddit community r/showerthoughts. \
        Vary the sentence structure between the different sentences, and try to be clever, creative, and funny. \
        The showerthoughts should be relatable and connected to things that people might encounter during mundane tasks."

In [ ]:
prompt_v2 = f"Please generate {number_of_showerthoughts} showerthoughts, which are inspired by the Reddit community r/showerthoughts. "+\
        "Do not repeat identical syntax between different sentences while only exchanging a single or few words. "+\
        "Be clever, creative, and (if suitable for the respective topic) funny. "+\
        "The showerthoughts should be relatable and connected to things that people might encounter during mundane tasks."
